In [2]:
from keras import optimizers, losses
from keras.layers import *
from keras.models import Model
from keras.backend import int_shape
from keras.utils import to_categorical, plot_model

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [38]:
def se_block(block_input, num_filters, ratio=8):
    pool1 = GlobalAveragePooling2D()(block_input)
    flat = Reshape((1, 1, num_filters))(pool1)
    dense1 = Dense(num_filters//ratio, activation='relu')(flat)
    dense2 = Dense(num_filters, activation='sigmoid')(dense1)
    scale = multiply([block_input, dense2])
    
    return scale

def resnet_block(block_input, num_filters):
    if int_shape(block_input)[3] != num_filters:
        block_input = Conv2D(num_filters, kernel_size=(1, 1))(block_input)
        
    conv1 = Conv2D(num_filters, kernel_size=(3, 3), padding='same')(block_input)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)
    conv2 = Conv2D(num_filters, kernel_size=(3, 3), padding='same')(relu1)
    norm2 = BatchNormalization()(conv2)
    relu2 = Activation('relu')(norm2)
    conv3 = Conv2D(num_filters, kernel_size=(3, 3), padding='same')(relu2)
    norm3 = BatchNormalization()(conv3)
    
    se = se_block(norm3, num_filters=num_filters)
    
    sum = Add()([block_input, se])
    relu3 = Activation('relu')(sum)
    
    return relu3

def resnet_8():
    for i in range (8):
        SE_output = resnet_block(relu3, num_filters = num_filters)
    return SE_output

def BLSTM():
    model = Sequential()
    model.add(Bidirectional(LSTM(512, return_sequences=True, merge_mode ='mul')))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    after_dp = Dropout(0.2)(model)
    model = Attention(units=32)(after_dp) 
    
    model = keras.models.Model(inputs=spectrogram,outputs=net)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    

In [1]:
def BLSTM():
    model = Sequential()
    model.add(Bidirectional(LSTM(512, return_sequences=True, merge_mode ='mul')))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    after_dp = Dropout(0.2)(model)
    model = Attention(units=32)(after_dp) 
    
    model = keras.models.Model(inputs=spectrogram,outputs=net)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    